# CC pipeline: Run the StarDist2D segmentation model over all folders

This notebook loads a pretrained StarDist2D segmentation model and applies the segmentation prediction on all folders within the masterfolder mainf (defined in 2nd code cell). Only microscopy chamber data containing folders should be within mainf. The segmentation is applied onto all images that end with *_PH.tif* and the segmentation image is saved into a newly created folder within each image folder named *seg_sd2*. For the moment, it assumes single-page tif files and saves single-page tif files with the exact same name as the input image used for segmentation prediction.

In [1]:
# directory
mainconfigname = 'config_example_cc';
configdir = 'C://Users/zinke/Documents/GitHub/microfluidics-image-processing/CC_pipeline/';

if not mainconfigname.endswith('.json'):
    mainconfigname += '.json'

import json
# Read JSON data
with open(configdir+mainconfigname, 'r') as file:
    data = json.load(file)

# Assign each key-value pair as a variable
for key, value in data.items():
    globals()[key] = value

In [2]:
import numpy as np
import sys
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from tifffile import imread, imwrite
from datetime import datetime
from csbdeep.utils import Path, normalize
from skimage.measure import regionprops, regionprops_table
from skimage import io
from skimage import segmentation
from skimage import color
from stardist.matching import matching_dataset
from stardist import fill_label_holes, random_label_cmap, relabel_image_stardist, calculate_extents, gputools_available, _draw_polygons
from stardist.models import Config2D, StarDist2D, StarDistData2D
import os
from tensorflow import keras
import cv2
import pandas as pd
from datetime import date
np.random.seed(42)
lbl_cmap = random_label_cmap()

def add_prefix(props, prefix):
    return {f"{prefix}_{key}" if 'intensity' in key else key: value for key, value in props.items()}

### Check if GPU can be accessed

In [3]:
gputools_available()

__init__.py (274): Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.


If you want to compute separable approximations, please install it with
pip install scikit-tensor-py3


True

### Load in meta file and display head. Check if correct

In [10]:
print(os.path.join(masterdir,metacsv))
meta = pd.read_csv(os.path.join(masterdir,metacsv), dtype={'stardist': str})
replicates = meta.replicate.unique()
meta.head()

E://Microscopy/Julian/sharing_cc/cc_example_meta_processing.csv


,Folder,date,replicate,chip,Process,pos,replicate2,Process2,rep2startdifferencemin,rep2firstframe,...,StageY,PxinUmX,PxinUmY,BarrierYincrop,chamberbox1,chamberbox2,chamberbox3,chamberbox4,repchip,prerotate
0,NaN,11/16/0024,c02,c1,609,1,NaN,NaN,NaN,NaN,...,-37618.36,0.065,0.065,727,740,233,747,1473,c02c1,NaN
1,NaN,11/16/0024,c02,c3,681,2,NaN,NaN,NaN,NaN,...,-38906.86,0.065,0.065,700,633,407,747,1473,c02c3,NaN
2,NaN,02/26/0025,c05,c1,2368,1,NaN,NaN,NaN,NaN,...,-35464.76,0.065,0.065,687,653,373,733,1473,c05c1,NaN
3,NaN,02/26/0025,c05,c3,2486,2,NaN,NaN,NaN,NaN,...,-37523.76,0.065,0.065,680,507,347,747,1467,c05c3,NaN


### Load stardist model
Here, the model is loaded. You need to specify the dir which contains a folder named *stardist* in the config file. This *stardist* folder needs to contain the files *weigths_best.h5* as well as the *config.json* and optionally the *thresholds.json*

In [11]:
print(stardistmodeldir)
model = StarDist2D(None, name='stardist', basedir=stardistmodeldir)
axis_norm = (0,1)   # normalize channels independently

C://Users/zinke/Documents/GitHub/microfluidics-image-processing/stardist_models/cc
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.493779, nms_thresh=0.3.


### Define regionprops parameters. You could add more if you want to

In [12]:
if flims:
    prop_list = ['label', 
                'area', 'centroid', 
                'axis_major_length', 'axis_minor_length',
                 'eccentricity',
                'intensity_mean', 'intensity_max']
else:
    prop_list = ['label', 
                'area', 'centroid', 
                'axis_major_length', 'axis_minor_length',
                 'eccentricity'] 

### Limit GPU RAM usage by StarDist

In [13]:
from csbdeep.utils.tf import limit_gpu_memory
# adjust as necessary: limit GPU memory to be used by TensorFlow to leave some to OpenCL-based computations
limit_gpu_memory(fraction=ramlimit, total_memory=ramsize)
# alternatively, try this:
# limit_gpu_memory(None, allow_growth=True)

Virtual devices cannot be modified after being initialized


## Main segmentation loop
This loop goes over each row in the meta file which is marked with completed preprocessing (Progress == 'Done') and applies the StarDist segmentation model to each position/chamber iteratively. For the moment, not paralellized but could probably benefit from that.

In [14]:
# Patch Keras model's predict to always use verbose=0
orig_predict = model.keras_model.predict
def predict_no_verbose(*args, **kwargs):
    kwargs['verbose'] = 0
    return orig_predict(*args, **kwargs)
model.keras_model.predict = predict_no_verbose

In [ ]:
for i in range(meta.shape[0]):
    # Skip if already processed or marked for exclusion
    if meta.loc[i, 'stardist'] == 'Done' or meta.loc[i, 'Exclude'] == 'excl':
        continue
        
    if not meta.loc[i, ('register')] == 'Done':
        continue
    # Directory setup for current experiment
    main_folder = os.path.join(masterdir, savedirname, meta.replicate[i], 'Chambers')
    save_directory = os.path.join(main_folder, 'stardistdata')
    os.makedirs(save_directory, exist_ok=True)

    current_directory = os.path.join(main_folder, f'Pos{str(meta.pos[i]).zfill(2)}_Chamb01')
    if not os.path.exists(current_directory):
        continue

    output_folder = os.path.join(current_directory, "seg_sd2")
    os.makedirs(output_folder, exist_ok=True)
    # Clear output folder if not empty
    for file in Path(output_folder).glob('*tif'):
        os.remove(file)

    # Collecting timelapse images
    images = sorted(Path(current_directory).glob('*Ch1*tif'))
    # Additional channels if applicable
    if flims:
        images_fl = sorted(Path(current_directory).glob('*Ch2*tif'))
        if n_channel > 2:
            images_fl2 = sorted(Path(current_directory).glob('*Ch3*tif'))

    # Determine frame range
    max_frame = meta.loc[i, 'MaxFr']
    frame_list = range(len(images)) if np.isnan(max_frame) else range(int(max_frame) - 1)
    
    # Process each frame
    fails = []  # To record any failures
    for frame_index in tqdm(frame_list, desc=f"{meta.replicate[i]}, pos {str(meta.pos[i]).zfill(2)}"):
        try:
            # Reading fluorescence images if available
            if flims:
                fluorescence_image = imread(images_fl[frame_index])
                if n_channel > 2:
                    fluorescence_image2 = imread(images_fl2[frame_index])
            
            # Main segmentation process
            main_image = imread(images[frame_index])
            normalized_image = normalize(main_image, 1, 99.8, axis=axis_norm)
            labels, details = model.predict_instances(normalized_image)
            
            # Save segmentation labels
            filename_segmentation = os.path.join(output_folder, os.path.basename(images[frame_index]))
            imwrite(filename_segmentation, labels, append=False, metadata=None)
            
            # Region properties calculation
            region_props = regionprops_table(labels, intensity_image=fluorescence_image if flims else None, properties=prop_list)
            if flims and n_channel > 2:
                region_props = add_prefix(region_props, 'fluor1')
                region_props2 = regionprops_table(labels, intensity_image=fluorescence_image2, properties=prop_list)
                region_props2 = add_prefix(region_props2, 'fluor2')
                # Merge intensity data for multiple fluorescence channels
                for key, value in region_props2.items():
                    if 'intensity' in key:
                        region_props[key] = value
            
            # Dataframe handling: compile and format region properties
            region_props_df = pd.DataFrame(region_props)
            region_props_df.insert(0, 'frame', frame_index + 1)
            region_props_df.insert(1, 'pos', int(os.path.basename(current_directory)[-2:]))
            region_props_df.insert(2, 'replicate', meta.replicate[i])
            region_props_df['folder'] = current_directory
            
            if frame_index == 0:
                all_frames_df = region_props_df
            else:
                all_frames_df = pd.concat([all_frames_df, region_props_df], ignore_index=True)
            
        except Exception as e:
            # Log any errors encountered during processing
            fails.append(f"Error processing folder {current_directory}, Frame {frame_index}: {e}")

    # Save compiled data for current position
    if 'all_frames_df' in locals():
        all_frames_df.to_csv(os.path.join(save_directory, os.path.basename(current_directory) + '.csv'), index=False)

    # Update metadata to indicate completion and log any failures
    meta = pd.read_csv(os.path.join(masterdir, metacsv), dtype={'stardist': str})
    meta.at[i, 'stardist'] = 'Done'
    if fails:
        meta.at[i, 'stardist_fails'] = '; '.join(fails)

    # Save metadata with updates
    meta.to_csv(os.path.join(masterdir, metacsv), index=False)

c02, pos 02:  84%|████████████████████████████████████████████████████████           | 163/195 [10:53<01:59,  3.75s/it]